In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from gensim.models import KeyedVectors
import matplotlib.font_manager as fm
from sklearn.manifold import TSNE
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import sys
from pathlib import Path
import sklearn

# 设置更好的可视化风格
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
sns.set_style("whitegrid")
# 假设notebooks目录在项目根目录下
project_root = Path.cwd().parent
sys.path.append(str(project_root))
from src.utils import download_chinese_font
import src.utils as utils
# 下载并安装字体
font_path = download_chinese_font()

# 设置matplotlib使用下载的字体
if font_path:
    plt.rcParams['font.sans-serif'] = ['SimHei'] + plt.rcParams['font.sans-serif']
    print("成功设置中文字体")
else:
    print("无法设置中文字体，将使用替代方案")

plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号



# 定义项目根目录和模型目录
PROJECT_ROOT = Path.cwd().parent
MODELS_DIR = PROJECT_ROOT / "models"

# 可能的模型目录
FINE_TUNED_MODELS_DIR = MODELS_DIR / "fine_tuned_vectors_flexible"
SLIDING_WINDOW_MODELS_DIR = MODELS_DIR / "fine_tuned_vectors_sliding_window"

# 检查哪个目录存在并包含模型
if FINE_TUNED_MODELS_DIR.exists() and any(FINE_TUNED_MODELS_DIR.glob("*_wordvectors.kv")):
    MODELS_DIR = FINE_TUNED_MODELS_DIR
    print(f"使用固定时期模型目录: {MODELS_DIR}")
elif SLIDING_WINDOW_MODELS_DIR.exists():
    # 查找滑动窗口模型的子目录
    subdirs = [d for d in SLIDING_WINDOW_MODELS_DIR.iterdir() if d.is_dir()]
    if subdirs:
        MODELS_DIR = subdirs[0]  # 使用第一个子目录
        print(f"使用滑动窗口模型目录: {MODELS_DIR}")
    else:
        print(f"滑动窗口模型目录存在，但没有子目录")
else:
    print(f"未找到模型目录，使用默认路径: {MODELS_DIR}")



# 加载模型
models = utils.load_models(MODELS_DIR=MODELS_DIR)

# 检查模型是否成功加载
if not models:
    print("没有成功加载任何模型，请检查模型路径")
else:
    print(f"\n成功加载了 {len(models)} 个模型:")
    for period_name, model in models.items():
        print(f"  {period_name}: 词汇量 {len(model.index_to_key)}")

字体已存在: /home/fangshikai/.fonts/SimHei.ttf
刷新字体缓存...
成功设置中文字体
使用固定时期模型目录: /home/fangshikai/law-word-vector/models/fine_tuned_vectors_flexible
找到 3 个模型文件:
  加载模型: Era1_1978-1996
  成功加载 Era1_1978-1996, 词汇量: 4874
  加载模型: Era2_1997-2013
  成功加载 Era2_1997-2013, 词汇量: 4992
  加载模型: Era3_2014-2024
  成功加载 Era3_2014-2024, 词汇量: 5000

成功加载了 3 个模型:
  Era1_1978-1996: 词汇量 4874
  Era2_1997-2013: 词汇量 4992
  Era3_2014-2024: 词汇量 5000


In [3]:
# 定义各时期词表文件路径
era_files = {
    'Era1_1978-1996': 'similar_words/Era1_1978-1996_final.txt',
    'Era2_1997-2013': 'similar_words/Era2_1997-2013_final.txt',
    'Era3_2014-2024': 'similar_words/Era3_2014-2024_final.txt'
}

# 读取所有时期的词表
similar_words_by_period = {}
for era, file_path in era_files.items():
    word_list = utils.load_expert_word_list(file_path)
    if word_list:
        print(f"已加载 {era} 词表，包含 {len(word_list)} 个词")
        similar_words_by_period[era] = word_list
    else:
        print(f"警告: {era} 词表为空")
# similar_words_by_period = update_sim
intersection, union, filtered_similar_words = utils.get_word_sets(similar_words_by_period, top_n=150, exclude_words_path="exclude_words.txt")
# similar_words_by_period_union = build_similar_words_by_period_from_word_list(models, "法治",union )


已加载 Era1_1978-1996 词表，包含 150 个词
已加载 Era2_1997-2013 词表，包含 150 个词
已加载 Era3_2014-2024 词表，包含 150 个词
已加载 17 个排除词
Era1_1978-1996: 过滤后保留 149 个词
Era2_1997-2013: 过滤后保留 148 个词
Era3_2014-2024: 过滤后保留 148 个词
所有时期共有词: 41 个
所有时期词汇并集: 284 个
